In [ ]:
from lissi_realsense import MyReader
import cv2
import opencv_jupyter_ui as jcv2
import lissilab
import random
import shutil
import os

pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoFake2/'
pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoReal/'
pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoFake/'

saveRealDepth = './realDepth.jpg'

reader = MyReader(pathToBag)

cap = cv2.VideoCapture(pathToBag + 'Color.avi')
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

randomlist = random.sample(range(length), 10)

real = 'real'
fake = 'fake'

realList = []
fakeList = []

for i in randomlist:
    reader._read_frame(i)
    color=reader.get_color_frame()
    depth=reader.get_depth_frame()
    infrared=reader.get_infrared_frame(1)
    depth_color=reader.get_colorized_depth()

    face_landmarks_list = lissilab.face_landmarks(color)

    chin01 = face_landmarks_list[0]['chin'][1]
    chin15 = face_landmarks_list[0]['chin'][15]
    nose_tip = face_landmarks_list[0]['nose_tip'][2]
    left_eye = face_landmarks_list[0]['left_eye'][5]
    right_eye = face_landmarks_list[0]['right_eye'][5]

    chinDistance01 = reader.measure_depth(chin01[0], chin01[1])
    chinDistance01 = "{:.2f}".format(chinDistance01)
    chinDistance15 = reader.measure_depth(chin15[0], chin15[1])
    chinDistance15 = "{:.2f}".format(chinDistance15)
    nose_tipDistance = reader.measure_depth(nose_tip[0], nose_tip[1])
    nose_tipDistance = "{:.2f}".format(nose_tipDistance)
    left_eyeDistance = reader.measure_depth(left_eye[0], left_eye[1])
    left_eyeDistance = "{:.2f}".format(left_eyeDistance)
    right_eyeDistance = reader.measure_depth(right_eye[0], right_eye[1])
    right_eyeDistance = "{:.2f}".format(right_eyeDistance)

    if chinDistance01 != left_eyeDistance and \
        chinDistance01 != nose_tipDistance and \
        chinDistance01 != right_eyeDistance and \
        chinDistance15 != left_eyeDistance and \
        chinDistance15 != nose_tipDistance and \
        chinDistance15 != right_eyeDistance and \
        nose_tipDistance != right_eyeDistance and \
        nose_tipDistance != left_eyeDistance:
        realList.append(real)
        realImage = color
        face_location = lissilab.face_locations(realImage)
        x, y, w, h = face_location[0]
        realImage = realImage[x-20:w, h:y]

    elif chinDistance01 == left_eyeDistance and \
        chinDistance01 == nose_tipDistance and \
        chinDistance01 == right_eyeDistance and \
        chinDistance15 == left_eyeDistance and \
        chinDistance15 == nose_tipDistance and \
        chinDistance15 == right_eyeDistance and \
        nose_tipDistance == right_eyeDistance and \
        nose_tipDistance == left_eyeDistance:
        fakeList.append(fake)

realDict = {i:realList.count(i) for i in realList}
fakeDict = {i:fakeList.count(i) for i in fakeList}

if realList and fakeList:
    if realDict[real] > fakeDict[fake]:
        print(real)
        cv2.imwrite(saveRealDepth, realImage)
    elif fakeDict[fake] > realDict[real]:
        print(fake)
    elif fakeDict[fake] == realDict[real]:
        print('Do another capture')
elif realList and not fakeList:
    print(real)
    cv2.imwrite(saveRealDepth, realImage)
elif fakeList and not realList:
    print(fake)

shutil.rmtree(pathToBag + 'Color')
shutil.rmtree(pathToBag + 'Depth')
os.remove(pathToBag + 'Color.avi')
os.remove(pathToBag + 'Depth.avi')
os.remove(pathToBag + 'meta.pkl')

In [ ]:
import dlib

print(dlib.DLIB_USE_CUDA)